# Natural Language Inference using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

In this lab we'll work with neural networks for natural language inference. Our task is: given a premise sentence P and hypothesis H, what entailment relationship holds between them? Is H entailed by P, contradicted by P or neutral towards P?

Given a sentence P, if H definitely describe something true given P then it is an **entailment**. If H describe something that's *maybe* true given P, it's **neutral**, and if H describe something that's definitely *false* given P it's a **contradiction**. 

# 1. Data

We will explore natural language inference using neural networks on the SNLI dataset, described in [1]. The dataset can be downloaded [here](https://nlp.stanford.edu/projects/snli/). We prepared a "simplified" version, with only the relevant columns [here](https://gubox.box.com/s/idd9b9cfbks4dnhznps0gjgbnrzsvfs4).

The (simplified) data is organized as follows (tab-separated values):
* Column 1: Premise
* Column 2: Hypothesis
* Column 3: Relation

Like in the previous lab, we'll use torchtext to build a dataloader. You can essentially do the same thing as you did in the last lab, but with our new dataset. **[1 mark]**

In [29]:
import string
import torch
device = torch.device('cuda:0')
import torch.nn as nn
import torchtext
import random
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import Field, BucketIterator, Iterator, TabularDataset
from IPython import embed
import numpy as np
import pandas as pd
import random



def dataloader(path_to_snli, batchsize):

    whitespacer = lambda x: x.split(' ')

    
    PREMISE = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True) # enforce the (batch, words) structure
    HYPOTHESIS = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True)
    RELATION = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True)

    train,test = TabularDataset.splits(path   = path_to_snli,
                                       train = 'train.csv',
                                       test = 'test.csv',
                            format = 'csv',
                            fields = [('premise', PREMISE),
                                    ('hypothesis', HYPOTHESIS),
                                     ('relation', RELATION)],
                            skip_header       = False,
                            csv_reader_params = {'delimiter':'\t',
                                                             'quotechar':'½'})
    # build vocabularies based on what our csv files contained and create word2id mapping
    PREMISE.build_vocab(train, test)
    HYPOTHESIS.vocab = PREMISE.vocab
    
    RELATION.build_vocab(train, test)

    # create batches from our data, and shuffle them for each epoch
    train_set, test_set = BucketIterator.splits((train,test),
                                batch_size        = batchsize,
                                sort_within_batch = True,
                                sort_key          = lambda x: len(x.premise),
                                shuffle           = True,
                                device            = device)

    return train_set, test_set, PREMISE, HYPOTHESIS, RELATION


# 2. Model

In this part, we'll build the model for predicting the relationship between H and P.

We will process each sentence using an LSTM. Then, we will construct some representation of the sentence. When we have a representation for H and P, we will combine them into one vector which we can use to predict the relationship.

We will train a model described in [2], the BiLSTM with max-pooling model. The procedure for the model is roughly:

    1) Encode the Hypothesis and the Premise using one shared bidirectional LSTM (or two different LSTMS)
    2) Perform max over the tokens in the premise and the hypothesis
    3) Combine the encoded premise and encoded hypothesis into one representation
    4) Predict the relationship 

### Creating a representation of a sentence

Let's first consider step 2 where we perform max/mean pooling. There is a function in pytorch for this, but we'll implement it from scratch. 

Let's consider the general case, what we want to do for these methods is apply some function $f$ along dimension $i$, and we want to do this for all $i$'s. As an example we consider the matrix S with size ``(N, D)`` where N is the number of words and D the number of dimensions:

$S = \begin{bmatrix}
    s_{11} & s_{12} & s_{13} & \dots  & s_{1d} \\
    s_{21} & s_{22} & s_{23} & \dots  & s_{2d} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    s_{n1} & s_{n2} & s_{n3} & \dots  & s_{nd}
\end{bmatrix}$

What we want to do is apply our function $f$ on each dimension, taking the input $s_{1d}, s_{2d}, ..., s_{nd}$ and generating the output $x_d$. 

You will implement both the max pooling method. When performing max-pooling, $max$ will be the function which selects a _maximum_ value from a vector and $x$ is the output, thus for each dimension $d$ in our output $x$ we get:

\begin{equation}
    x_d = max(s_{1d}, s_{2d}, ..., s_{nd})
\end{equation}


This operation will reduce a batch of size ``(batch_size, num_words, dimensions)`` to ``(batch_size, dimensions)`` meaning that we now have created a sentence representation based on the content of the words representations in the sentence. 

Create a function that takes as input a tensor of size ``(batch_size, num_words, dimensions)`` then performs max pooling and returns the result (the output should be of size: ```(batch_size, dimensions)```). [**4 Marks**]

In [31]:
def max_tensor(tense): #takes 2-dimensional vector and selects the max for each index of the embedding dim
    new_tensor = torch.zeros(tense.size()[1])
    for i in range(tense.shape[1]):
        new_tensor[i] = max(tense[:,i])
    return new_tensor
     
def pooling(input_tensor):
    output_tensor = torch.zeros((input_tensor.size()[0],input_tensor.size()[2])) #size of output tensor will be batch_size by dimensions
    for i in range(input_tensor.size()[0]): #for i in the range of the numer of sentences/representations, in this case probably the batch length i guess yeww
        max_pool = max_tensor(input_tensor[i,:,:]) #for the item/sentence-thingy 'i' in our batch, get the max embedding value for each embedding (Out of all the words in the sentence)
        output_tensor[i] = max_pool
    return output_tensor

### Combining sentence representations

Next, we need to combine the premise and hypothesis into one representation. We will do this by concatenating four tensors (the final size of our tensor $X$ should be ``(batch_size, 4d)`` where ``d`` is the number of dimensions that you use): 

$$X = [P; H; |P-H|; P \cdot H]$$

Here, what we do is concatenating P, H, P times H, and the absolute value of P minus H, then return the result.

Implement the function. **[2 marks]**

In [32]:
def combine_premise_and_hypothesis(hypothesis, premise):
    subtracted = torch.sub(premise, hypothesis)
    multiplied = torch.mul(premise,hypothesis)
    output = torch.cat((premise, hypothesis, subtracted, multiplied), 1)   
    return output

### Creating the model

Finally, we can build the model according to the procedure given previously by using the functions we defined above. Additionaly, in the model you should use *dropout*. For efficiency purposes, it's acceptable to only train the model with either max or mean pooling. 

Implement the model [**6 marks**]

In [33]:
class SNLIModel(nn.Module):
    def __init__(self, num_words, emb_dim, num_labels):
        super(SNLIModel, self).__init__()
        self.embeddings = nn.Embedding(num_words, emb_dim)
        self.LSTM = nn.LSTM(emb_dim, emb_dim, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(emb_dim*8, num_labels)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, premise, hypothesis):
        p = self.embeddings(premise)
        h = self.embeddings(hypothesis)
        p_lstm, _ = self.LSTM(p)
        h_lstm, _ = self.LSTM(h)

        p_pooled = pooling(p_lstm)
        h_pooled = pooling(h_lstm)
        
        ph_representation = combine_premise_and_hypothesis(h_pooled, p_pooled)
        predictions = self.classifier(ph_representation)
        
        return predictions

# 3. Training and testing

As before, implement the training and testing of the model. SNLI can take a very long time to train, so I suggest you only run it for one or two epochs. **[2 marks]** 

**Tip for efficiency:** *when developing your model, try training and testing the model on one batch (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [34]:
hype = {
'batch_size' : 8,
'learning_rate' : 0.001,
'epochs' : 3,
'emb_dim' : 32,
'output_dim' : 32
}

train_set, test_set, PREMISE, HYPOTHESIS, RELATION = dataloader("./simple_snli_1/", hype['batch_size'])

In [ ]:
loss_function = nn.CrossEntropyLoss()

model = SNLIModel(
        num_words = len(PREMISE.vocab.itos),
        emb_dim = hype['emb_dim'], 
        num_labels = len(RELATION.vocab.itos)
        )

model.to(device)

optimizer = optim.Adam(
            model.parameters(),
            lr=hype['learning_rate'])

model.train()
total_loss = 0

for epoch in range(hype['epochs']):
    for i, batch in enumerate(train_set):

        premise = batch.premise
        hypothesis = batch.hypothesis
        relation = batch.relation
        output = model(premise, hypothesis)
        loss = loss_function(output, relation.reshape(-1))
        total_loss += loss.item()
        
        print(total_loss/(i+1), end='\r')
        
        loss.backward()
        
        optimizer.step()
        
        optimizer.zero_grad()

In [ ]:
torch.save(model.state_dict(), "SNLIModel.model")

In [ ]:
model = SNLIModel(
        num_words = len(PREMISE.vocab.itos),
        emb_dim = hype['emb_dim'], 
        num_labels = len(RELATION.vocab.itos)
        )
model.load_state_dict(torch.load("SNLIModel.model"))
model = model.to(device)

# Testing the model:

In [ ]:
test_loss = 0   
model.eval()

total = 0
correct = 0

per_relation_total = dict.fromkeys(RELATION.vocab.itos, 0)
per_relation_correct = dict.fromkeys(RELATION.vocab.itos, 0)

for i, batch in enumerate(test_set):     
    relations = batch.relation
    premise = batch.premise
    hypothesis = batch.hypothesis
    
    with torch.no_grad(): 
        output = model(premise, hypothesis)
        loss = loss_function(output, relations.reshape(-1))       
        test_loss += loss.item()
        print('>', np.round(test_loss/(i+1), 4), end='\r')
        
        # Calculate total accuracy
        total += relations.size(0)
        predicted = [RELATION.vocab.itos[x] for x in torch.max(output, 1)[1]]
        true_relations = [RELATION.vocab.itos[x] for x in relations]
        
        for n in range(len(predicted)):
            if predicted[n] == true_relations[n]:
                correct += 1
                
        # Calculate per-word-form accuracy
        for n,relation in enumerate(true_relations):
            if predicted[n] == true_relations[n]:
                per_relation_correct[relation] += 1
            per_relation_total[relation] += 1

In [ ]:
accuracy1 = correct / total
accuracy2 = {true_relation : f'{np.round((per_relation_correct[true_relation] / per_relation_total[true_relation]) * 100,2)} %' for true_relation in true_relations if per_relation_total[true_relation] > 0}
print(f'Total accuracy: {np.round(accuracy1 * 100, 2)} %')

print('Per-relation accuracy:')
for k,v in accuracy2.items():
    print(f'{k} : {v}')

Suggest a _baseline_ that we can compare our model against **[2 marks]**

- We could choose the most probable inference from the set we are testing on. 
- If all the examples in the data are equally probable, we could just assume a 1/total_classes, so if there are 3 classes, that would be a 33.33% probability for any. 

Suggest some ways (other than using a baseline) in which we can analyse the models performance **[4 marks]**.

- Check how well it performs for each relation, in terms of recall, accuracy and F1 score as well as create a confusion matrix. 
- Testing how it performs on different datasets, and compare performance. If it performs comparably on other datasets, it would show that this can generalise. 
- Test different parameters for the model and see how the performance changes with the parameter changes. 
- Compare it to other machine learning models, such as Bert. 

Suggest some ways to improve the model **[3 marks]**.

- By improving consideration of context, word order, word class etc. Perhaps running this on Bert would be a good way of doing this. 
- Looking at examples that it gets wrong, and see if there are some observable generalisations or patterns that would help us to make design improvements to the model. 
- Run the model for more epochs, use more data. 
- Use a validation set to help us select the best parameters for our model; especially avoiding overfitting. 

### Readings

[1] Samuel R. Bowman, Gabor Angeli, Christopher Potts, and Christopher D. Manning. 2015. A large annotated corpus for learning natural language inference. In Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing (EMNLP). 

[2] Conneau, A., Kiela, D., Schwenk, H., Barrault, L., & Bordes, A. (2017). Supervised learning of universal sentence representations from natural language inference data. arXiv preprint arXiv:1705.02364.